In [2]:
import pandas as pd
from bs4 import BeautifulSoup

In [3]:
with open("../vexere_trips_raw_sample.html", "r", encoding="utf-8") as file:
    content = file.read()

FileNotFoundError: [Errno 2] No such file or directory: '../vexere_trips_raw_sample.html'

In [ ]:
def extract_rating_from_container(container):
    try:
        ratings = []
        rate_divs = container.find_all('div', class_='rate-title')
        for rate_div in rate_divs:
            rate_ps = rate_div.find_all('p')
            if len(rate_ps) >= 2:
                rate_title = rate_ps[0].get_text(strip=True)
                rate_point = rate_ps[1].get_text(strip=True)
                ratings.append((rate_title, rate_point))
        if ratings:
            return ratings
        else:
            return [(None, None)]
    except Exception:
        return [(None, None)]

In [ ]:
def specify_price_elements(block) -> bool:
    fare = block.find('div', class_='fare')      #  chỉ hiển thị 1 giá     
    
    if fare: # khong co gia khuyen mai
        return True
    
    return False

def parse_fare_small(block):

    fare_small = block.find('div', class_='fareSmall')
    
    original_price = fare_small.find('div', class_='small').get_text(strip=True).replace("đ", "").strip()
    
    if fare_small.find('div', class_='percent'):
        percent_discount = fare_small.find('div', class_='percent').get_text(strip=True)
    else:
        percent_discount = None
        
    return original_price, percent_discount

def parser_fare_sale(block):
    fare_sale = block.find('div', class_='fare-sale')
    original_price, percent_discount = parse_fare_small(block)
    
    if fare_sale and fare_sale.get_text(strip=True):
        sale_price = fare_sale.get_text(strip=True).replace("đ", "").replace('Từ ', '').strip()
        
    return original_price, sale_price, percent_discount

def parse_fare(block):
    fare = block.find('div', class_='fare')
    fare_price = fare.get_text(strip=True).replace("đ", "").replace('Từ ', '').strip()
    
    original_price = fare_price
    sale_price = None
    percent_discount = None
    
    return original_price ,sale_price, percent_discount  

def parse_price(block):
    if specify_price_elements(block):
        return parse_fare(block)
    else:
        return parser_fare_sale(block)

In [ ]:
def parse_bus_info(container):
    bus_name = container.find('div', class_='bus-name').text.strip() if container.find('div', class_='bus-name') else None
    bus_rating = container.find('div', class_='bus-rating').find('span').text.strip() if container.find('div', class_='bus-rating') and container.find('div', class_='bus-rating').find('span') else None
    seat_type = container.find('div', class_='seat-type').get_text(strip=True) if container.find('div', class_='seat-type') else None

    from_to_content = container.find('div', class_="from-to-content")
    if from_to_content:
        to_content = from_to_content.find('div', class_='content to')
        from_content = from_to_content.find('div', class_='content from')
        duration = from_to_content.find('div', class_="duration").get_text(strip=True) if from_to_content.find('div', class_="duration") else None

        # Arrival info
        date_arrival = None
        to_hour = None
        to_place = None
        if to_content:
            span = to_content.find('span', class_="text-date-arrival-time")
            date_arrival = span.get_text(strip=True) if span else None
            content_to_info = to_content.find('div', class_='content-to-info')
            if content_to_info:
                to_hour = content_to_info.find('div',class_='hour' ).get_text(strip=True) if content_to_info.find('div',class_='hour' ) else None
                to_place = content_to_info.find('div',class_='place' ).get_text(strip=True) if content_to_info.find('div',class_='place' ) else None

        # Departure info
        from_hour = from_content.find('div',class_='hour' ).get_text(strip=True) if from_content and from_content.find('div',class_='hour' ) else None
        from_place = from_content.find('div',class_='place' ).get_text(strip=True) if from_content and from_content.find('div',class_='place' ) else None
    else:
        duration = None
        date_arrival = None
        to_hour = None
        to_place = None
        from_hour = None
        from_place = None

    price_original, price_discount, discount_percent = parse_price(container)

    notification = container.find('div', class_='link')
    notification = notification.get_text(strip=True) if notification else None

    return [
        bus_name, bus_rating, seat_type,
        from_hour, from_place, duration,
        to_hour, to_place, date_arrival,
        price_original, price_discount, discount_percent, notification
    ]

In [ ]:
def extract_all_bus_info(soup) -> list:
    containers = soup.find_all("div", class_="container")
    bus_info_list = [parse_bus_info(container) for container in containers]
    return bus_info_list

In [ ]:
# soup = BeautifulSoup(content, 'html.parser')
# data = extract_all_bus_info(soup)
# df_1 = pd.DataFrame(data)

In [ ]:
def parse_price(block):
    fare_sale = block.find('div', class_='fare-sale')
    fare = block.find('div', class_='fare')
    small = block.find('div', class_='small')
    percent = block.find('div', class_='percent')

    # Discounted price
    if fare_sale:
        price_discount = fare_sale.get_text(strip=True).replace("Từ ", "").replace("đ", "").strip()
    elif fare:
        price_discount = fare.get_text(strip=True).replace("đ", "").strip()
    else:
        price_discount = None

    # Original price
    if small and small.get_text(strip=True):
        price_original = small.get_text(strip=True).replace("đ", "").strip()
    else:
        price_original = price_discount

    # Discount percent
    discount_percent = percent.get_text(strip=True) if percent else None

    return [price_original, price_discount, discount_percent]

In [ ]:
# soup = BeautifulSoup(content, 'html.parser')
# data = extract_all_bus_info(soup)
# df_2 = pd.DataFrame(data)

In [ ]:
# with open('new_crawl_data.html', 'r', encoding='utf-8') as file:
#     content = file.read()

In [ ]:
# soup = BeautifulSoup(content, 'html.parser')

In [ ]:
all_trips_info = []
columns_trips_info = [
    'bus_company', 'bus_star', 'bus_type',
    'departure_time', 'pick_up_point', 'duration_h_m',
    'arrival_time', 'drop_off_point', 'arrival_date',
    'price_original', 'price_discount', 'discount_percent', 'notification',
]

containers = soup.find_all("div", class_="container")
for container in containers:
    bus_info = parse_bus_info(container)
    ratings = extract_rating_from_container(container)
    
    # --- Xử lý df_ratings ---
    df_ratings = pd.DataFrame(ratings, columns=['rate_title', 'rate_point']).T
    header_row = df_ratings.iloc[0] 
    df_ratings.columns = header_row
    
    
    df_ratings = df_ratings[1:]
    
    df_ratings = df_ratings.reset_index(drop=True)
    
    # --- Xử lý df_bus_info ---
    df_bus_info = pd.DataFrame([bus_info], columns=columns_trips_info)
    
   
    df_bus_info = df_bus_info.reset_index(drop=True)

    # --- Nối ngang ---
    
    trip_info = pd.concat([df_bus_info, df_ratings], axis=1)
    all_trips_info.append(trip_info)

all_trips_info = pd.concat(all_trips_info, ignore_index=True)
all_trips_info.to_csv("../data/bus_info_with_rating_new.csv", index=False)
all_trips_info

,bus_company,bus_star,bus_type,departure_time,pick_up_point,duration_h_m,arrival_time,drop_off_point,arrival_date,price_original,price_discount,discount_percent,notification,None
0,HK BUSLINES,4.6 (1774),Limousine giường phòng 22 chỗ (WC),17:01,• Văn phòng Hạ Long,3h30m,20:31,• NINH BÌNH - Bến Thuyền Tam Cốc,None,Từ 250.000,Từ 250.000,None,None,None
1,Tam Cốc Limousine,4.2 (18),Limousine 9 chỗ,12:30,• Tùng Luxury Hotel Hạ Long,3h40m,16:10,• VP Ninh Bình,None,240.000,230.000,-4%,None,None
2,Ninh Bình Travel,4.2 (66),Limousine 11 chỗ,12:00,• Tùng Luxury Hotel Hạ Long,4h,16:00,• Văn phòng Ninh Bình,None,350.000,332.500,-5%,None,None
3,Ninh Bình Excursion Transport,4.2 (166),Limousine 11 chỗ VIP,12:01,• Văn phòng Quảng Ninh,3h,15:01,• Văn phòng Ninh Bình,None,270.000,260.000,-4%,Chính sách trẻ em,None
4,Quang Mười,4 (120),Limousine 9 chỗ (P),13:40,• Hạ Long - Bến xe Bãi Cháy,3h20m,17:00,• Ninh Bình - Bến xe,None,250.000,240.000,-4%,Thông Báo Nhà Xe,None
5,The New Way,3.1 (28),Limousine 9 chỗ,13:00,• VP Hạ Long,3h20m,16:20,• Mike's Travel Agency - Bến Thuyền Tam Cốc,None,290.000,290.000,None,None,None
6,Duy Khánh Limousine,3.8 (653),Limousine 9 chỗ,17:50,"• Số 24 Thủy Tùng, KS Tùng Luxury",2h50m,20:40,• Bến Xe Điện Tam Cốc,None,250.000,240.000,-4%,None,None
7,Green Bus (Quảng Ninh),2.8 (4),Limousine 10 chỗ,19:00,• Văn phòng Hạ Long,3h,22:00,• Mike's Travel Agency,None,300.000,300.000,None,None,None
8,Đức Duy Bus,0 (0),Limousine 34 phòng,14:30,• Tùng Luxury Hotel Hạ Long,2h,16:30,• BUSBOOKING Ninh Bình,None,350.000,350.000,None,None,None
9,Ninh Bình Excursion Transport,4.2 (166),Limousine 23 chỗ,12:02,• Văn phòng Quảng Ninh,3h,15:02,• Văn phòng Ninh Bình,None,230.000,220.000,-4%,Chính sách trẻ em,None


In [ ]:
df = pd.read_csv("../data/sg_nt_11102025.csv")

In [ ]:
df_routes = pd.read_csv("../data/routes.csv")

In [ ]:
df

,bus_company,bus_star,bus_type,departure_time,pick_up_point,duration_h_m,arrival_time,drop_off_point,arrival_date,price_original,...,discount_percent,notification,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ,Unnamed: 20
0,Đà Lạt ơi,4.8 (3425),Limousine 24 Phòng ĐÔI,23:30,• Trạm Hàng Xanh,7h,06:30,• Trạm Nha Trang,(12/10),450.000,...,-29%,Lộ trình: N4 Thủ Đức - Quận 1 - Cao tốc,4.8,4.8,4.8,4.8,4.8,4.8,4.8,NaN
1,Bình Minh Tải,4.7 (3650),Limousine 22 Phòng Đơn,22:30,• Văn Phòng Quận 1,7h5m,05:35,• Văn phòng Nha Trang,(12/10),350.000,...,-14%,Hướng đi: Cao tốc (Long Thành - Dầu Giây - Pha...,4.6,4.6,4.7,4.6,4.6,4.6,4.8,NaN
2,Huỳnh Gia,4.7 (8534),Giường nằm 38 chỗ (WC),22:30,• Văn Phòng Phạm Ngũ Lão,6h30m,05:00,• Văn Phòng Nha Trang,(12/10),280.000,...,-11%,Lộ trình: Cao tốc (Long Thành - Dầu Giây - Pha...,4.8,4.7,4.7,4.7,4.6,4.6,4.7,NaN
3,An Anh Limousine,4.8 (8322),Limousine 34 Phòng Đơn,23:30,• Văn Phòng Quận 5,6h30m,06:00,• Văn phòng Nha Trang,(12/10),299.000,...,-33%,Tiện ích miễn phí,4.8,4.7,4.7,4.7,4.7,4.7,4.6,NaN
4,Khanh Phong,4.7 (16845),Limousine 20 giường phòng (WC),05:15,• Văn Phòng Phạm Ngũ Lão - Quận 1.,6h10m,11:25,• Văn Phòng Nha Trang (KS Mường Thanh),NaN,480.000,...,-17%,Quý khách lưu ý,4.8,4.7,4.7,4.7,4.6,4.6,4.8,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,Phúc An Express,4.7 (3425),Limousine 22 giường phòng (WC),22:30,• Văn Phòng Sài Gòn,7h40m,06:10,• Chợ Vĩnh Lương,(12/10),480.000,...,NaN,Lộ trình xe chạy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
561,Phúc An Express,4.7 (3425),Limousine 22 giường phòng (WC),22:45,• Trạm Phúc An Express,7h15m,06:00,• Văn Phòng Nha Trang,(12/10),470.000,...,NaN,Lộ trình xe chạy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
562,Phúc An Express,4.7 (3425),Limousine 22 giường phòng (WC),22:45,• Trạm Phúc An Express,7h25m,06:10,• Chợ Vĩnh Lương,(12/10),480.000,...,NaN,Lộ trình xe chạy,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
563,Quỳnh Nhật,3.7 (279),Limousine 24 phòng,10:00,• Bến xe Miền Đông mới,8h5m,18:05,• Ngã 3 Thành.,NaN,650.000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_routes

,start_point,destination_1,destination_2,destination_3,destination_4,destination_5,destination_6,destination_7,destination_8,destination_9,destination_10
0,Hà Nội,Hải Phòng,Nghệ An,Sơn La,Hà Giang,Quảng Ninh,Thanh Hóa,SaPa,Ninh Bình,NaN,NaN
1,Quảng Ninh,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
2,Ninh Bình,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
3,Đà Nẵng,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
4,Sài Gòn,Bình Thuận,Ninh Thuận,Đắk Lắk,Gia Lai,Phú Yên,Nha Trang,Bà Rịa - Vũng Tàu,NaN,NaN,NaN
5,Sa Pa,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
6,Vũng Tàu,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
7,Đà Lạt,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
8,Nha Trang,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
9,Phan Thiết,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết


In [ ]:
df.drop(columns=[df.columns[-1]], inplace = True, axis = 1)

In [ ]:
df

,bus_company,bus_star,bus_type,departure_time,pick_up_point,duration_h_m,arrival_time,drop_off_point,arrival_date,price_original,price_discount,discount_percent,notification,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ
0,Đà Lạt ơi,4.8 (3425),Limousine 24 Phòng ĐÔI,23:30,• Trạm Hàng Xanh,7h,06:30,• Trạm Nha Trang,(12/10),450.000,320.000,-29%,Lộ trình: N4 Thủ Đức - Quận 1 - Cao tốc,4.8,4.8,4.8,4.8,4.8,4.8,4.8
1,Bình Minh Tải,4.7 (3650),Limousine 22 Phòng Đơn,22:30,• Văn Phòng Quận 1,7h5m,05:35,• Văn phòng Nha Trang,(12/10),350.000,300.000,-14%,Hướng đi: Cao tốc (Long Thành - Dầu Giây - Pha...,4.6,4.6,4.7,4.6,4.6,4.6,4.8
2,Huỳnh Gia,4.7 (8534),Giường nằm 38 chỗ (WC),22:30,• Văn Phòng Phạm Ngũ Lão,6h30m,05:00,• Văn Phòng Nha Trang,(12/10),280.000,250.000,-11%,Lộ trình: Cao tốc (Long Thành - Dầu Giây - Pha...,4.8,4.7,4.7,4.7,4.6,4.6,4.7
3,An Anh Limousine,4.8 (8322),Limousine 34 Phòng Đơn,23:30,• Văn Phòng Quận 5,6h30m,06:00,• Văn phòng Nha Trang,(12/10),299.000,199.000,-33%,Tiện ích miễn phí,4.8,4.7,4.7,4.7,4.7,4.7,4.6
4,Khanh Phong,4.7 (16845),Limousine 20 giường phòng (WC),05:15,• Văn Phòng Phạm Ngũ Lão - Quận 1.,6h10m,11:25,• Văn Phòng Nha Trang (KS Mường Thanh),NaN,480.000,400.000,-17%,Quý khách lưu ý,4.8,4.7,4.7,4.7,4.6,4.6,4.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,Phúc An Express,4.7 (3425),Limousine 22 giường phòng (WC),22:30,• Văn Phòng Sài Gòn,7h40m,06:10,• Chợ Vĩnh Lương,(12/10),480.000,480.000,NaN,Lộ trình xe chạy,NaN,NaN,NaN,NaN,NaN,NaN,NaN
561,Phúc An Express,4.7 (3425),Limousine 22 giường phòng (WC),22:45,• Trạm Phúc An Express,7h15m,06:00,• Văn Phòng Nha Trang,(12/10),470.000,470.000,NaN,Lộ trình xe chạy,NaN,NaN,NaN,NaN,NaN,NaN,NaN
562,Phúc An Express,4.7 (3425),Limousine 22 giường phòng (WC),22:45,• Trạm Phúc An Express,7h25m,06:10,• Chợ Vĩnh Lương,(12/10),480.000,480.000,NaN,Lộ trình xe chạy,NaN,NaN,NaN,NaN,NaN,NaN,NaN
563,Quỳnh Nhật,3.7 (279),Limousine 24 phòng,10:00,• Bến xe Miền Đông mới,8h5m,18:05,• Ngã 3 Thành.,NaN,650.000,650.000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
import pandas as pd 
import requests 
from bs4 import BeautifulSoup

In [ ]:
URL = "https://vexere.com/vi-VN/ve-xe-khach-tu-quang-ninh-di-ninh-binh-149t1421.html?date=11-10-2025&v=6&nation=84"

In [ ]:
response = requests.get(URL)
soup = BeautifulSoup(response.content, 'html.parser')

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time, random
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:

driver = webdriver.Chrome()
driver.get(URL)

# Open all ratings after all trips are loaded
try:
    stars = WebDriverWait(driver, 15).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".bus-rating-button .anticon-star"))
    )
    for star in stars:
        driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", star)
        time.sleep(1)  # tạm nghỉ nhẹ để trang ổn định
        ActionChains(driver).move_to_element(star).click().perform()
        time.sleep(2)
except Exception as e:
    print("⚠️ Error clicking star icons:", e)

time.sleep(3)
soup = BeautifulSoup(driver.page_source, "html.parser")

all_trips_info = []
columns_trips_info = [
    'bus_company', 'bus_star', 'bus_type',
    'departure_time', 'pick_up_point', 'duration_h_m',
    'arrival_time', 'drop_off_point', 'arrival_date',
    'price_original', 'price_discount', 'discount_percent', 'notification',
]

containers = soup.find_all("div", class_="container")
for container in containers:
    bus_info = parse_bus_info(container)
    ratings = extract_rating_from_container(container)
    
    # --- Xử lý df_ratings ---
    df_ratings = pd.DataFrame(ratings, columns=['rate_title', 'rate_point']).T
    header_row = df_ratings.iloc[0] 
    df_ratings.columns = header_row
    
    
    df_ratings = df_ratings[1:]
    
    df_ratings = df_ratings.reset_index(drop=True)
    
    # --- Xử lý df_bus_info ---
    df_bus_info = pd.DataFrame([bus_info], columns=columns_trips_info)
    
   
    df_bus_info = df_bus_info.reset_index(drop=True)

    # --- Nối ngang ---
    
    trip_info = pd.concat([df_bus_info, df_ratings], axis=1)
    all_trips_info.append(trip_info)

all_trips_info = pd.concat(all_trips_info, ignore_index=True)
# all_trips_info.to_csv("../data/bus_info_with_rating.csv", index=False)
driver.quit()

In [ ]:
all_trips_info

,bus_company,bus_star,bus_type,departure_time,pick_up_point,duration_h_m,arrival_time,drop_off_point,arrival_date,price_original,...,discount_percent,notification,An toàn,Thông tin chính xác,Thông tin đầy đủ,Thái độ nhân viên,Tiện nghi & thoải mái,Chất lượng dịch vụ,Đúng giờ,None
0,HK BUSLINES,4.6 (1778),Limousine giường phòng 22 chỗ (WC),17:01,• Văn phòng Hạ Long,3h30m,20:31,• NINH BÌNH - Bến Thuyền Tam Cốc,None,Từ 250.000,...,None,None,4.7,4.6,4.6,4.6,4.6,4.6,4.7,NaN
1,Tam Cốc Limousine,4.2 (18),Limousine 9 chỗ,12:30,• Tùng Luxury Hotel Hạ Long,3h40m,16:10,• VP Ninh Bình,None,240.000,...,-4%,None,4.4,3.6,3.6,4.4,4.2,4.1,5.0,NaN
2,Ninh Bình Travel,4.2 (66),Limousine 11 chỗ,12:00,• Tùng Luxury Hotel Hạ Long,4h,16:00,• Văn phòng Ninh Bình,None,350.000,...,-5%,None,4.1,4.2,4.4,4.5,4.0,4.3,NaN,NaN
3,Ninh Bình Excursion Transport,4.2 (166),Limousine 11 chỗ VIP,12:01,• Văn phòng Quảng Ninh,3h,15:01,• Văn phòng Ninh Bình,None,270.000,...,-4%,Chính sách trẻ em,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
4,Quang Mười,4 (120),Limousine 9 chỗ (P),13:40,• Hạ Long - Bến xe Bãi Cháy,3h20m,17:00,• Ninh Bình - Bến xe,None,250.000,...,-4%,Thông Báo Nhà Xe,4.1,4.0,4.0,4.0,4.0,4.0,4.4,NaN
5,The New Way,3.1 (28),Limousine 9 chỗ,13:00,• VP Hạ Long,3h20m,16:20,• Mike's Travel Agency - Bến Thuyền Tam Cốc,None,290.000,...,None,None,3.3,3.2,3.0,3.0,3.3,3.2,5.0,NaN
6,Duy Khánh Limousine,3.8 (654),Limousine 9 chỗ,17:50,"• Số 24 Thủy Tùng, KS Tùng Luxury",2h50m,20:40,• Bến Xe Điện Tam Cốc,None,250.000,...,-4%,None,4.0,4.0,4.1,4.0,3.9,3.7,4.0,NaN
7,Green Bus (Quảng Ninh),2.8 (4),Limousine 10 chỗ,19:00,• Văn phòng Hạ Long,3h,22:00,• Mike's Travel Agency,None,300.000,...,None,None,3.3,3.0,3.0,3.5,3.3,3.3,1.0,NaN
8,Đức Duy Bus,0 (0),Limousine 34 phòng,14:30,• Tùng Luxury Hotel Hạ Long,2h,16:30,• BUSBOOKING Ninh Bình,None,350.000,...,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
9,Ninh Bình Excursion Transport,4.2 (166),Limousine 23 chỗ,12:02,• Văn phòng Quảng Ninh,3h,15:02,• Văn phòng Ninh Bình,None,230.000,...,-4%,Chính sách trẻ em,4.4,4.2,4.1,4.3,4.3,4.3,4.5,NaN


In [4]:
import pandas as pd

In [12]:
df_route = pd.read_csv("../../data/routes.csv")
df_route

,start_point,destination_1,destination_2,destination_3,destination_4,destination_5,destination_6,destination_7,destination_8,destination_9,destination_10
0,Hà Nội,Hải Phòng,Nghệ An,Sơn La,Hà Giang,Quảng Ninh,Thanh Hóa,SaPa,Ninh Bình,NaN,NaN
1,Quảng Ninh,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
2,Ninh Bình,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
3,Đà Nẵng,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
4,Sài Gòn,Bình Thuận,Ninh Thuận,Đắk Lắk,Gia Lai,Phú Yên,Nha Trang,Bà Rịa - Vũng Tàu,NaN,NaN,NaN
5,Sa Pa,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
6,Vũng Tàu,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
7,Đà Lạt,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
8,Nha Trang,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết
9,Phan Thiết,Hà Nội,Quảng Ninh,Ninh Bình,Đà Nẵng,Sài Gòn,Sa Pa,Vũng Tàu,Đà Lạt,Nha Trang,Phan Thiết


In [13]:
df_route = df_route['start_point']
df_route

0        Hà Nội
1    Quảng Ninh
2     Ninh Bình
3       Đà Nẵng
4       Sài Gòn
5         Sa Pa
6      Vũng Tàu
7        Đà Lạt
8     Nha Trang
9    Phan Thiết
Name: start_point, dtype: object

In [15]:
df_route = df_route.reset_index(drop=True)
df_route

0        Hà Nội
1    Quảng Ninh
2     Ninh Bình
3       Đà Nẵng
4       Sài Gòn
5         Sa Pa
6      Vũng Tàu
7        Đà Lạt
8     Nha Trang
9    Phan Thiết
Name: start_point, dtype: object

In [ ]:
df_route = pd.DataFrame(df_route)


In [22]:
df_route.shape

(10, 1)

In [24]:
df_route['city_abbr '] = df_route['start_point'].apply(lambda x: ''.join([word[0].upper() for word in x.split()]))
df_route

,start_point,ten_tat,city_abbr
0,Hà Nội,HN,HN
1,Quảng Ninh,QN,QN
2,Ninh Bình,NB,NB
3,Đà Nẵng,ĐN,ĐN
4,Sài Gòn,SG,SG
5,Sa Pa,SP,SP
6,Vũng Tàu,VT,VT
7,Đà Lạt,ĐL,ĐL
8,Nha Trang,NT,NT
9,Phan Thiết,PT,PT
